
# Singular Value Decomposition (SVD)

SVD is a matrix factorization technique useful in signal processing, statistics, and machine learning.

Given a matrix \( X \in \mathbb{R}^{m \times n} \), the SVD is given by:

$$
X = U \Sigma V^T
$$

Where:
- \( U \in \mathbb{R}^{m \times m} \) is an orthogonal matrix (left singular vectors)
- \( \Sigma \in \mathbb{R}^{m \times n} \) is a diagonal matrix with non-negative singular values
- \( V \in \mathbb{R}^{n \times n} \) is an orthogonal matrix (right singular vectors)


In [ ]:

import numpy as np

# Example matrix
X = np.random.rand(5, 3)
U, s, VT = np.linalg.svd(X, full_matrices=False)
print("U shape:", U.shape)
print("s:", s)
print("VT shape:", VT.shape)



# From SVD to PCA

PCA can be derived from the SVD of the centered data matrix. Let \( X \) be the data matrix, and \( X_{\text{mean}} \) be the mean of the columns.

We project the data into the principal components using:

$$
\Phi = U^T (X - \bar{X})
$$

To visualize:

```python
Phi = np.matmul(U.transpose(), X - X_mean[:, None])
```


In [ ]:

import matplotlib.pyplot as plt

X_mean = X.mean(axis=1)
Phi = np.matmul(U.T, X - X_mean[:, None])
fig, ax = plt.subplots()
ax.scatter(Phi[0, :], Phi[1, :])
ax.set_aspect('equal')
plt.title("PCA Projection")
plt.show()


In [ ]:

# Binary classification using projection on PC_1
threshold = 0.0
digits = [0, 1]
labels_test = np.random.choice(digits, X.shape[1])
A_test = X
A_mean = X.mean(axis=1)

PC_1 = np.matmul(U[:, 0].T, (A_test - A_mean[:, None]))

labels_predicted = np.empty(labels_test.shape).astype(int)
labels_predicted[PC_1 > threshold] = digits[0]
labels_predicted[PC_1 <= threshold] = digits[1]

true_0 = np.sum(np.logical_and(labels_test == digits[0], labels_predicted == digits[0]))
false_0 = np.sum(np.logical_and(labels_test == digits[1], labels_predicted == digits[0]))
true_1 = np.sum(np.logical_and(labels_test == digits[1], labels_predicted == digits[1]))
false_1 = np.sum(np.logical_and(labels_test == digits[0], labels_predicted == digits[1]))

print(f"true  {digits[0]}: {true_0}")
print(f"false {digits[0]}: {false_0}")
print(f"true  {digits[1]}: {true_1}")
print(f"false {digits[1]}: {false_1}")
accuracy = (true_0 + true_1) / (true_0 + true_1 + false_0 + false_1)
print(f"accuracy = {accuracy * 100:.2f} %")



# Least Squares Regression

We consider the linear model:

$$
y = mx + q
$$

Where:
- \( m = 2 \), \( q = 3 \)
- \( x_i \sim \mathcal{N}(0, 1) \)
- \( \epsilon_i \sim \mathcal{N}(0, 2^2) \)
- \( \tilde{y}_i = y_i + \epsilon_i \)

We want to recover \( m \) and \( q \) using the least squares method.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Generate data
N = 100
m_true = 2
q_true = 3
sigma = 2

x = np.random.randn(N)
y = m_true * x + q_true
noise = np.random.normal(0, sigma, size=N)
y_tilde = y + noise

# Plot
plt.scatter(x, y_tilde, label='Noisy Data', alpha=0.7)
plt.plot(np.sort(x), m_true * np.sort(x) + q_true, color='red', label='True Line')
plt.legend()
plt.title("Linear Model with Noise")
plt.xlabel("x")
plt.ylabel("y")
plt.grid(True)
plt.show()


In [ ]:

# Least Squares using Moore-Penrose Pseudoinverse
X_design = np.vstack((x, np.ones_like(x))).T
theta = np.linalg.pinv(X_design) @ y_tilde

# Predicted line
x_pred = np.linspace(x.min(), x.max(), 100)
y_pred = theta[0] * x_pred + theta[1]

# Plot
plt.scatter(x, y_tilde, label='Noisy Data', alpha=0.7)
plt.plot(x_pred, y_pred, label='Least Squares Fit', color='green')
plt.plot(np.sort(x), m_true * np.sort(x) + q_true, color='red', label='True Line')
plt.legend()
plt.title("Least Squares Fit")
plt.grid(True)
plt.show()



# Nonlinear Regression and Kernel Methods

We now consider the function:

$$
f(x) = \tanh(2x - 1)
$$

We'll perform standard regression and then explore different kernel methods.


In [ ]:

# Generate data
N = 100
x = np.random.randn(N)
y = np.tanh(2 * x - 1)
noise = np.random.normal(0, 0.1, size=N)
y_tilde = y + noise

# Test data
x_test = np.linspace(-3, 3, 1000)
y_test_true = np.tanh(2 * x_test - 1)

# Plot
plt.scatter(x, y_tilde, label='Noisy Train Data', alpha=0.5)
plt.plot(x_test, y_test_true, label='True Function', color='black')
plt.title("Nonlinear Target Function")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Kernel Regression: Polynomial and Gaussian
def kernel_matrix(X1, X2, kernel_fn):
    return np.array([[kernel_fn(xi, xj) for xj in X2] for xi in X1])

def poly_kernel(q):
    return lambda x, y: (x * y + 1)**q

def gaussian_kernel(sigma):
    return lambda x, y: np.exp(-(x - y)**2 / (2 * sigma**2))

def kernel_regression(x_train, y_train, x_test, kernel_fn):
    K = kernel_matrix(x_train, x_train, kernel_fn)
    alpha = np.linalg.solve(K, y_train)
    K_test = kernel_matrix(x_test, x_train, kernel_fn)
    return K_test @ alpha

# Apply kernels
k_poly = poly_kernel(q=2)
k_gauss = gaussian_kernel(sigma=0.5)

y_pred_poly = kernel_regression(x, y_tilde, x_test, k_poly)
y_pred_gauss = kernel_regression(x, y_tilde, x_test, k_gauss)

# Plot results
plt.plot(x_test, y_test_true, label='True Function', color='black')
plt.plot(x_test, y_pred_poly, label='Poly Kernel (q=2)', color='blue')
plt.plot(x_test, y_pred_gauss, label='Gaussian Kernel (σ=0.5)', color='orange')
plt.scatter(x, y_tilde, label='Train Data', alpha=0.3)
plt.legend()
plt.title("Kernel Regression")
plt.grid(True)
plt.show()



# Support Vector Regression (SVR)

SVR aims to find a function that deviates from the actual data points by a value no greater than \( \epsilon \), while also being as flat as possible.

The SVR loss function is defined as:

$$
L(w) = \lambda \|w\|^2 + \frac{1}{n} \sum_{i=1}^{n} \max(0, |f(x_i) - y_i| - \epsilon)
$$

Where:
- \( \lambda \): regularization parameter
- \( \epsilon \): tolerance margin
- \( f(x_i) \): predicted output


In [ ]:

import jax
import jax.numpy as jnp

class SVR:
    def __init__(self, lmbda=1.0, epsilon=0.1):
        self.lmbda = lmbda
        self.epsilon = epsilon
        self.w = None

    def loss(self, params, X, y):
        pred = jnp.dot(X, params)
        loss_val = jnp.maximum(0, jnp.abs(pred - y) - self.epsilon)
        reg_term = self.lmbda * jnp.sum(params**2)
        return reg_term + jnp.mean(loss_val)

    def train(self, X, y):
        n_features = X.shape[1]
        self.w = jnp.zeros(n_features)

        opt_res = jax.scipy.optimize.minimize(self.loss, self.w, method="BFGS", args=(X, y))
        self.w = opt_res.x

    def predict(self, X):
        return jnp.dot(X, self.w)



# Support Vector Machine (SVM)

SVM is a supervised learning model used for classification. It tries to find a hyperplane that separates the classes with the maximum margin.

### Hinge Loss Function

$$
L(w) = \lambda \|w\|^2 + \frac{1}{n} \sum_{i=1}^{n} \max(0, 1 - y_i (w^T x_i + b))
$$

Where:
- \( \lambda \): regularization parameter
- \( y_i \in \{-1, +1\} \)


In [ ]:

class SVM:
    def __init__(self, lmbda=1.0):
        self.lmbda = lmbda
        self.w = None

    def loss(self, params, X, y):
        decision = jnp.dot(X, params[:-1]) + params[-1]
        loss_val = jnp.maximum(0, 1 - y * decision)
        reg_term = self.lmbda * jnp.sum(params[:-1] ** 2)
        return reg_term + jnp.mean(loss_val)

    def train(self, X, y):
        _, n_features = X.shape
        self.w = jnp.zeros(n_features + 1)

        opt_res = jax.scipy.optimize.minimize(self.loss, self.w, method="BFGS", args=(X, y))
        self.w = opt_res.x

    def predict(self, X):
        decision = jnp.dot(X, self.w[:-1]) + self.w[-1]
        return jnp.sign(decision)
